In [1]:
!git clone https://github.com/lessw2020/mish.git

Cloning into 'mish'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 8), reused 10 (delta 3), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [8]:
!git clone https://github.com/mgrankin/over9000.git

Cloning into 'over9000'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 71 (delta 37), reused 42 (delta 19), pack-reused 0
Unpacking objects: 100% (71/71), done.


In [1]:
%cd mish

/content/mish


In [2]:
from mxresnet import *

Mish activation loaded...
Mish activation loaded...


In [0]:
import torch, math
from torch.optim.optimizer import Optimizer
import itertools as it

class Ralamb(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(Ralamb, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(Ralamb, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Ralamb does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]

                if state['step'] == buffered[0]:
                    N_sma, radam_step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        radam_step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        radam_step_size = 1 / (1 - beta1 ** state['step'])
                    buffered[2] = radam_step_size

                update = torch.zeros_like(p_data_fp32)
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    update.addcdiv_(radam_step_size, exp_avg, denom)
                else:
                    update.add_(radam_step_size, exp_avg)

                if group['weight_decay'] != 0:
                    update.add_(group['weight_decay'], p_data_fp32)

                radam_norm = update.pow(2).sum().sqrt()
                weight_norm = p.data.pow(2).sum().sqrt()
                if weight_norm == 0 or radam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / radam_norm

                trust_ratio = max(0, min(10, trust_ratio))

                state['weight_norm'] = weight_norm
                state['adam_norm'] = radam_norm
                state['trust_ratio'] = trust_ratio

                p_data_fp32.add_(-update * trust_ratio * group['lr'])
                p.data.copy_(p_data_fp32)

        return loss

In [0]:
from fastai.vision import *

In [0]:
path = untar_data(URLs.IMAGEWOOF)

In [0]:
data = (ImageList.from_folder(path).split_by_folder(valid='val')
            .label_from_folder().transform(([flip_lr(p=0.5)], []), size=128)
            .databunch(bs=64, num_workers=2)
            .presize(128, scale=(0.35,1))
            .normalize(imagenet_stats))

In [0]:
opt_func=partial(Ralamb, betas = (0.9,0.99), eps=1e-6)

In [0]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *

In [0]:
lr = 1e-2

In [0]:
res = []
num_epoch=5
learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=opt_func,
               metrics=[accuracy, top_k_accuracy],
               bn_wd=False, true_wd=True,
               loss_func=LabelSmoothingCrossEntropy())
n = len(learn.data.train_dl)
anneal_start = int(n*5*0.7)
phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
phases = [phase0, phase1]
sched = GeneralScheduler(learn, phases)
learn.callbacks.append(sched)
learn.fit(num_epoch)

loss, acc, topk = learn.validate()
res.append(acc.numpy())

In [13]:
print(res)

[array(0.532, dtype=float32)]


In [0]:
learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=opt_func,
               metrics=[accuracy, top_k_accuracy],
               bn_wd=False, true_wd=True,
               loss_func=LabelSmoothingCrossEntropy())
n = len(learn.data.train_dl)
anneal_start = int(n*5*0.7)
phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
phases = [phase0, phase1]
sched = GeneralScheduler(learn, phases)
learn.callbacks.append(sched)
learn.fit(num_epoch)

loss, acc, topk = learn.validate()
res.append(acc.numpy())

In [0]:
learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=opt_func,
               metrics=[accuracy, top_k_accuracy],
               bn_wd=False, true_wd=True,
               loss_func=LabelSmoothingCrossEntropy())
n = len(learn.data.train_dl)
anneal_start = int(n*5*0.7)
phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
phases = [phase0, phase1]
sched = GeneralScheduler(learn, phases)
learn.callbacks.append(sched)
learn.fit(num_epoch)

loss, acc, topk = learn.validate()
res.append(acc.numpy())

In [0]:
learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=opt_func,
               metrics=[accuracy, top_k_accuracy],
               bn_wd=False, true_wd=True,
               loss_func=LabelSmoothingCrossEntropy())
n = len(learn.data.train_dl)
anneal_start = int(n*5*0.7)
phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
phases = [phase0, phase1]
sched = GeneralScheduler(learn, phases)
learn.callbacks.append(sched)
learn.fit(num_epoch)

loss, acc, topk = learn.validate()
res.append(acc.numpy())

In [0]:
learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=opt_func,
               metrics=[accuracy, top_k_accuracy],
               bn_wd=False, true_wd=True,
               loss_func=LabelSmoothingCrossEntropy())
n = len(learn.data.train_dl)
anneal_start = int(n*5*0.7)
phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
phases = [phase0, phase1]
sched = GeneralScheduler(learn, phases)
learn.callbacks.append(sched)
learn.fit(num_epoch)

loss, acc, topk = learn.validate()
res.append(acc.numpy())